In [7]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

In [8]:
from PyKakao import *
from config import KEYS
api_key = KEYS.get("KAKAO")

In [9]:
API = DaumSearch(api_key)

In [11]:
query = "카카오"
df = API.search_cafe(query, dataframe=True)
df

total_count: 2252303, pageable_count: 842, is_end: False


,cafename,contents,datetime,thumbnail,title,url
0,도탁스 (DOTAX),니트 시계 코도리 맥주 모자 뱃살 힙색 새다리 스타킹 변기통 머그컵 문어 가면 모자 우산,2022-12-01T22:45:41.000+09:00,https://search4.kakaocdn.net/argon/130x130_85_...,친구 생일 때 선물하면 좋은 <b>카카오</b>톡 선물 모음,http://cafe.daum.net/dotax/OUBq/18794
1,우리 동네 목욕탕,《 우리 동네 목욕탕 / 아이셔요 》 출처 https://www.daum.net/n...,2022-10-16T09:21:03.000+09:00,,16일 8시 기준 현재 <b>카카오</b> 관련 어플 복구 상황,http://cafe.daum.net/weareshower/ZEmx/139314
2,＊여성시대＊ 차분한 20대들의 알흠다운 공간,출처 : https://m.sportsw.kr/news/newsview.php?nc...,2022-12-06T11:50:58.000+09:00,https://search4.kakaocdn.net/argon/130x130_85_...,"[<b>카카오</b>엔터] 1.6억뷰 ‘비밀 사이’, 드라마 제작확정...",http://cafe.daum.net/subdued20club/ReHf/4160991
3,방석 위로 모여라,[<b>카카오</b>톡] 1. 이모티콘 플러스 구독자 2. 기프티콘으로 맛있는 거 ...,2022-10-16T11:05:22.000+09:00,,<b>카카오</b> 서버터져서 생긴 피해를 모아보자,http://cafe.daum.net/Duckgu/D49M/244465
4,＊여성시대＊ 차분한 20대들의 알흠다운 공간,출처 : 여성시대 무너곰 <b>카카오</b>뱅크 26주적금 with 교촌치킨 htt...,2022-11-22T10:41:19.000+09:00,,<b>카카오</b>뱅크×교촌치킨 26주 적금 챌린지 (11.22~12.05),http://cafe.daum.net/subdued20club/LxCT/315657
5,우마무스메 프리티 더비 공식 카페,안녕하세요. <b>카카오</b>게임즈 대표이사 조계현입니다. ‘우마무스메 프리티 더...,2022-09-18T21:05:33.000+09:00,https://search1.kakaocdn.net/argon/130x130_85_...,<b>카카오</b>게임즈 대표이사 조계현입니다.,http://cafe.daum.net/umamusume-kor/Z4os/59
6,이종격투기,반갑습니다 게시판지기입니다 이번 <b>카카오</b> 계정 통합으로 인해 뜻하지 않게...,2022-08-31T19:24:51.000+09:00,https://search1.kakaocdn.net/argon/130x130_85_...,＜＜＜ <b>카카오</b> 계정 통합으로 인한 정회원 복구 신청란...,http://cafe.daum.net/ssaumjil/EpQf/375722
7,【우수카페】신비한 약초세상,<b>카카오</b> 판교 서버 화재로 어제 오후부터 카페 카스 카톡 페북등 오류로 ...,2022-10-16T05:50:54.000+09:00,,[경북방 출석부]10월16일(일)<b>카카오</b> 오류로 불편하시죠?,http://cafe.daum.net/sansamtkdgh/E1zX/31082
8,도탁스 (DOTAX),출처 링크 있었던거같은데 까먹음... 아마 이미지하단에 적혀있는 그것일수도,2022-10-05T17:46:48.000+09:00,https://search2.kakaocdn.net/argon/130x130_85_...,<b>카카오</b> 라이언+춘식이 배경화면 가져가세욧,http://cafe.daum.net/dotax/Elgq/4032446
9,도탁스 (DOTAX),ㅋㅋㅋㅋㅋ,2022-10-17T14:13:40.000+09:00,https://search1.kakaocdn.net/argon/130x130_85_...,영화) <b>카카오</b>:[화재],http://cafe.daum.net/dotax/FGFP/20783
